# Scoring script playground

## Main script for running queries

In [2]:
import requests
import random
import json

# URL     = 'https://dev.api.chat.ask.eduworks.com/webhooks/rest/webhook'
URL     = 'http://localhost:5005/webhooks/rest/webhook'

DATA    = {
    "message"   : "How do you treat peach leaf curl?",
    "sender"    : str(random.randint(0, 100000))
}

r = requests.post(URL, json = DATA)

if r.status_code == 200:
    print(json.loads(r.text))
else:
    print('Service unavailable')

[{'recipient_id': '4409', 'text': 'Extracted slots</br>[Format: (<i>relation</i>) <strong>entity</strong> - <strong>value</strong>]:</br>Group 1:</br>(<i>pest</i>) <strong>name</strong> - <strong>peach leaf curl</strong></br>(<i>pest</i>) <strong>action</strong> - <strong>treat</strong></br></br>Composed 1 additional queries:</br>1) <i>peach leaf curl treat</i></br>'}, {'recipient_id': '4409', 'text': 'Results with slots improvement... Top 10 results.'}, {'recipient_id': '4409', 'custom': {'text': 'Here are my top results:', 'payload': 'collapsible', 'data': [{'title': '<p>1)<em>Peach Leaf Curl</em></br>(score: 0.89)</br>(source: <a href="http://ipm.ucanr.edu/PMG/PESTNOTES/pn7426.html?src=exchbt" target="_blank">pestsIPM</a>)</p>', 'description': '<p><strong>Details</strong>: Peach leaf curl, also known as leaf curl, is a disease caused by the fungus Taphrina deformans. Peac</p></br><p><strong>Development</strong>: Peach leaf curl first appears in spring as reddish areas on developing 

In [3]:
import random
import re

sender = random.randint(0, 100000)

# URL     = 'https://dev.api.chat.ask.eduworks.com/webhooks/rest/webhook'
URL     = 'http://localhost:5005/webhooks/rest/webhook'
DATA    = {
    "message"   : "", 
    "sender"    : str(sender)
}
def run_tests(questions):
    
    results = []
    for i, q in enumerate(questions):

        DATA['message'] = q
        try:
            response = requests.post(URL, json = DATA)
            if response.status_code != 200:
                print(f'Error: Service at {URL} is unavailable, exit.')
                exit()

        except Exception as e:
            print(f'Error: Exception at posting question - "{q}", exit. {type(e).__name__}: "{e}".')
            exit()

        
        try:
            r = json.loads(response.text)
            success = False
            for link in r:
                if 'custom' in link:
                    success = True
                    r = link['custom']
                    break
        except Exception as e:
            print(f'Error: Failed on parsing response on question - "{q}", exit. {type(e).__name__}: "{e}".')
            exit()

        
        result = []
        if success:
           
            try:
                r = r['data']
                if len(r) == 0:
                    raise Exception
            except Exception as e:
                print(f'Error: Failed on parsing response on question - "{q}", exit. . {type(e).__name__}: "{e}".')

            for e in r:
                title   = re.findall("<em>(.*?)</em>"           , e['title'])[0]
                link    = re.findall("href=[\"\'](.*?)[\"\']"   , e['title'])[0]
                result.append((title, link))
            

            DATA['message'] = '/intent_affirm'
            try:
                response = requests.post(URL, json = DATA)
                if response.status_code != 200:
                    print(f'Error: Service at {URL} is unavailable, exit.')
                    exit()
            except Exception as e:
                print(f'Error: Exception at posting affirmative message on question - "{q}", exit. {type(e).__name__}: "{e}".')
                exit()
            
            try:
                r = json.loads(response.text)
                r = r[0]
                r = r['text']
                if 'Anything else I can help with?' != r:
                    raise Exception
            except Exception as e:
                print(f'Error: Failed on parsing response of affirmative message on question - "{q}", exit. {type(e).__name__}: "{e}".')
                exit()
        else:
            print(f'No results for question - "{q}"')
            # result.append(('', ''))
        
        if (i+1)%5 == 0:
            print(f'Finished {i+1} questions...')
        
        results.append(result)
    
    print(f'Finished querying all ({len(questions)}) questions for scoring')

    return results


## Loading questions

### Valid questions stats

In [4]:
import pandas as pd

VALID_DATA  = './data/transformed/valid_questions.pkl'

df = pd.read_pickle(VALID_DATA)
print(f'Shape of NA questions: {df.shape}')
df.sample(10)

Shape of NA questions: (115, 4)


,Question,Pest,URL,Source
47,I have a garden full of weeds. How do I clear ...,weeds in garden,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7441.html],UCIPM
37,What insecticide gets rid of whiteflies?,whiteflies,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7401.html],UCIPM
28,Is it better to plant asparagus in spring or s...,Vegetable growing,[http://ipm.ucanr.edu/PMG/GARDEN/VEGES/CULTURA...,UCIPM
36,We have this fly that is very tiny and black. ...,,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn74167.html],AE
49,We have small caterpillars that have defoliate...,,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7422.htm...,AE
4,What are the big black bugs on my patio at night?,Oriental cockroach,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7467.html],UCIPM
36,How do you set a gopher trap?,pocket gophers,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7433.htm...,UCIPM
18,What is biting me in bed?,mysterious bites,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7454.htm...,UCIPM
18,I have a bunch of aphids on our rose bushes. W...,,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7404.html],AE
40,Where do ground squirrels live?,ground squirrel,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7438.html],UCIPM


In [5]:
answers_pest    = df['Pest'     ].values.tolist()
answers_url     = df['URL'      ].values.tolist()
questions       = df['Question' ].values.tolist()
results         = run_tests(questions)

Finished 5 questions...
Finished 10 questions...
Finished 15 questions...
Finished 20 questions...
Finished 25 questions...
Finished 30 questions...
Finished 35 questions...
Finished 40 questions...
Finished 45 questions...
Finished 50 questions...
No results for question - "I started noticing some sort of bug that made webs in my oak tree. It seemed to be getting into the little branches and killing the rest of the branch. I am hoping you might know what this is and how to rectify. I am attaching a picture. "
Finished 55 questions...
Finished 60 questions...
Finished 65 questions...
Finished 70 questions...
Finished 75 questions...
Finished 80 questions...
Finished 85 questions...
Finished 90 questions...
Finished 95 questions...
Finished 100 questions...
Finished 105 questions...
Finished 110 questions...
Finished 115 questions...
Finished querying all (115) questions for scoring


In [6]:
scores = []
for i, r in enumerate(results):
    answer = answers_url[i]
    topn = [False, False, False, False]
    for i1, r1 in enumerate(r):
        if r1[1].split('?')[0] in answer:
            if i1 == 0:
                topn[0] = True
            if i1 < 3:
                topn[1] = True
            if i1 < 5:
                topn[2] = True
            topn[3] = True

    scores.append(topn)

In [7]:
top1    = 0
top3    = 0
top5    = 0
top10   = 0
for topn in scores:
    if topn[0]: top1    += 1
    if topn[1]: top3    += 1
    if topn[2]: top5    += 1
    if topn[3]: top10   += 1

print(f'Out of {len(results)} results, following correct:'  )
print(f'Top 1 : {top1 :<3d} ({top1 /len(results) * 100:<.2f}%)')
print(f'Top 3 : {top3 :<3d} ({top3 /len(results) * 100:<.2f}%)')
print(f'Top 5 : {top5 :<3d} ({top5 /len(results) * 100:<.2f}%)')
print(f'Top 10: {top10:<3d} ({top10/len(results) * 100:<.2f}%)')

Out of 115 results, following correct:
Top 1 : 41  (35.65%)
Top 3 : 53  (46.09%)
Top 5 : 60  (52.17%)
Top 10: 71  (61.74%)


### Valid questions errors

In [8]:
for i, score in enumerate(scores):
    if not any(score):
        print(f'------------------------------------------------------------------')
        print(f'Question number     : {i+1}')
        print(f'Question            : {questions[i]}')
        print(f'The title answers   : {answers_pest[i]}')
        print(f'The URL answers     : {answers_url[i]}')
        for i1, res in enumerate(results[i]):
            print(f'Results ranked {i1+1:<2}   : {res[0][:30] + "...":<33} {"(URL: " + res[1]:>10s})')

------------------------------------------------------------------
Question number     : 5
Question            : What are the big black bugs on my patio at night?
The title answers   : Oriental cockroach
The URL answers     : ['http://ipm.ucanr.edu/PMG/PESTNOTES/pn7467.html']
Results ranked 1    : Stink bugs...                     (URL: http://ipm.ucanr.edu/PMG/GARDEN/VEGES/PESTS/stinkbug.html?src=exchbt)
Results ranked 2    : Lygus bugs...                     (URL: http://ipm.ucanr.edu/PMG/GARDEN/VEGES/PESTS/lygusbugs.html?src=exchbt)
Results ranked 3    : Harlequin bug...                  (URL: http://ipm.ucanr.edu/PMG/GARDEN/VEGES/PESTS/harlequinbug.html?src=exchbt)
Results ranked 4    : Conenose Bugs...                  (URL: http://ipm.ucanr.edu/PMG/PESTNOTES/pn7455.html?src=exchbt)
Results ranked 5    : Squash Bugs...                    (URL: http://ipm.ucanr.edu/PMG/PESTNOTES/pn74144.html?src=exchbt)
Results ranked 6    : Black vine weevil...              (URL: http://ipm.ucanr.

In [9]:
error_results = {
    'question_number'   : [],
    'question_text'     : [],
    'answer_title'      : [],
    'answer_url'        : [],
    'result_urls'       : []
}

for i in range(10):
    error_results['result_urls'].append([])


for i, score in enumerate(scores):
    if not any(score):
        error_results['question_number' ].append(i+1            )
        error_results['question_text'   ].append(questions[i]   )
        error_results['answer_title'    ].append(answers_pest[i])
        error_results['answer_url'      ].append(', '.join(answers_url[i]))
        
        r_temp = ['No answer'] * 10
        for i1, res in enumerate(results[i]):
            r_temp[i1] = f'{res[0][:30] + "...":<33} {"(URL: " + res[1]:>10s})'
        for i1, res in enumerate(r_temp):
            error_results['result_urls'][i1].append(res)


In [10]:
data = {
    'QuestionNumber': error_results['question_number'   ],
    'QuestionText'  : error_results['question_text'     ],
    'AnswerTitle'   : error_results['answer_title'      ],
    'AnswerURL'     : error_results['answer_url'        ],
    'ResultRank1'   : error_results['result_urls'][0],
    'ResultRank2'   : error_results['result_urls'][1],
    'ResultRank3'   : error_results['result_urls'][2],
    'ResultRank4'   : error_results['result_urls'][3],
    'ResultRank5'   : error_results['result_urls'][4],
    'ResultRank6'   : error_results['result_urls'][5],
    'ResultRank7'   : error_results['result_urls'][6],
    'ResultRank8'   : error_results['result_urls'][7],
    'ResultRank9'   : error_results['result_urls'][8],
    'ResultRank10'  : error_results['result_urls'][9]
}

df_errors = pd.DataFrame(data = data).set_index('QuestionNumber')
print(f'Shape of error answers dataframe: {df_errors.shape}')
df_errors.sample(10)

Shape of error answers dataframe: (44, 13)


,QuestionText,AnswerTitle,AnswerURL,ResultRank1,ResultRank2,ResultRank3,ResultRank4,ResultRank5,ResultRank6,ResultRank7,ResultRank8,ResultRank9,ResultRank10
QuestionNumber,,,,,,,,,,,,,
22,Why is my tree dying?,tree decline/death,http://ipm.ucanr.edu/PMG/PESTNOTES/pn74109.htm...,Tree Borers... (URL: http:/...,Golden-chain tree... (URL: http:/...,Katsura tree... (URL: http:/...,Camphor tree... (URL: http:/...,Coral tree... (URL: http:/...,Leafrollers... (URL: http:/...,Sudden Oak Death... (URL: http:/...,Why is my Redwood tree dying?... (URL: https:...,Is my redwood tree dying?... (URL: https:...,My birch tree died.... (URL: https:...
14,My rose leaves are covered with dusty orange s...,rust on roses,http://ipm.ucanr.edu/PMG/PESTNOTES/pn7463.html,Orange rust... (URL: http:/...,Citrus rust mite (silver mite)... (URL: http:/...,Rust... (URL: http:/...,Rust... (URL: http:/...,Rust... (URL: http:/...,Yellow rust... (URL: http:/...,White pine blister rust... (URL: http:/...,Chrysanthemum rusts... (URL: http:/...,Rusts - Various fungi... (URL: http:/...,Botrytis leafspot (leaf blight... (URL: http:/...
59,Found very large white grub looking worm in fi...,,http://ipm.ucanr.edu/PMG/GARDEN/PLANTS/INVERT/...,just want to be sure.... (URL: https:...,Masked chafers (white grubs)... (URL: http:/...,Burning nettle... (URL: http:/...,Termites... (URL: http:/...,Drywood Termites... (URL: http:/...,Fire Blight... (URL: http:/...,No answer,No answer,No answer,No answer
81,I have a problem with fruit flies laying their...,,http://ipm.ucanr.edu/PMG/PESTNOTES/pn74158.html,Seedcorn maggot... (URL: http:/...,Cabbage maggot... (URL: http:/...,Gall midges... (URL: http:/...,Apple maggot... (URL: http:/...,Raspberry cane maggot... (URL: http:/...,Needle and twig midges... (URL: http:/...,Walnut Husk Fly... (URL: http:/...,Carrot rust fly... (URL: http:/...,Flies... (URL: http:/...,Fungus Gnats... (URL: http:/...
84,I found a green worm crawling out of my partia...,,http://ipm.ucanr.edu/home-and-landscape/tomato...,Boxwood leafminer... (URL: http:/...,Tomato pinworm... (URL: http:/...,California pear sawfly... (URL: http:/...,Potato tuberworm... (URL: http:/...,Pest identification and confir... (URL: http:/...,Redhumped Caterpillar... (URL: http:/...,Pea leaf weevil... (URL: http:/...,Tomato russet mite... (URL: http:/...,Tobacco mosaic virus... (URL: http:/...,Damping-off Diseases in the Ga... (URL: http:/...
78,How do I get rid of flea beetle infestation on...,,http://ipm.ucanr.edu/PMG/GARDEN/VEGES/PESTS/fl...,Squash bug... (URL: http:/...,Gall midges... (URL: http:/...,"Soap - fungicide, insecticide,... (URL: http:/...",Poplar and willow borer... (URL: http:/...,Vegetable weevil... (URL: http:/...,Pyrethrin - insecticide (botan... (URL: http:/...,"Leaf beetles, cucumber beetles... (URL: http:/...",Roses: Insects and Mites... (URL: http:/...,Lace Bugs... (URL: http:/...,Squash Bugs... (URL: http:/...
20,What causes tomatoes to wilt?,tomato wilt,http://ipm.ucanr.edu/home-and-landscape/tomato...,Verticillium wilt on tomatoes... (URL: http:/...,Fusarium wilt on tomatoes... (URL: http:/...,Sunscald on tomatoes... (URL: http:/...,Tomato spotted wilt virus... (URL: http:/...,Watering tomatoes... (URL: http:/...,Watering tomatoes... (URL: http:/...,Planting tomatoes... (URL: http:/...,Catfacing on tomatoes... (URL: http:/...,Fruit cracks on tomatoes... (URL: http:/...,Early blight on tomatoes... (URL: http:/...
68,How do I rid the tree of these brownish rust c...,,http://ipm.ucanr.edu/PMG/PESTNOTES/pn74158.html,Lygus bugs... (URL: http:/...,Rusts - Various fungi... (URL: http:/...,Pear rust mite... (URL: http:/...,Cypress tipminer... (URL: http:/...,Chrysanthemum rusts... (URL: http:/...,Bark Beetles... (URL: http:/...,Common causes of plant galls... (URL: http:/...,Chinch and false chinch bugs... (URL: http:/...,Roses: Insects and Mites... (URL: http:/...,Fungal diseases... (URL: http:/...
13,What does rust on roses look like?,rust on roses,http://ipm.ucanr.e

In [11]:
ERRORS_VALID_DATA = './data/errors/errors_valid.csv'
df_errors.to_csv(ERRORS_VALID_DATA)

### NA questions stats

In [12]:
import pandas as pd

NA_DATA     = './data/transformed/na_questions.pkl'

df = pd.read_pickle(NA_DATA)
print(f'Shape of NA questions: {df.shape}')
df.sample(10)

Shape of NA questions: (37, 2)


,Question,Source
65,We would like to prepare the soil for a vegeta...,AE
78,We have two raised beds and want to add some a...,AE
71,What cultivar would you recommend for a good t...,AE
81,Have we had our last frost for spring of 2022?...,AE
58,I have a black mold looking fungus growing on ...,AE
75,Do you know an affordable source to get evergr...,AE
72,I HAVE A 13 YEAR OLD ROSEBUSH I NEED TO MOVE D...,AE
63,I'm looking for a planting guide on current we...,AE
87,The trees on my property have long branches th...,AE
103,Is winter a good time to transplant my fruit t...,AE


In [13]:
questions       = df['Question'].values.tolist()
results         = run_tests(questions)

Finished 5 questions...
Finished 10 questions...
No results for question - "I'm a 4-H alumni. What is the nearest club near me? If there isn't one close, can I start my own club?"
No results for question - "I'm interested in hiring a master gardener from the master gardener program. Do you have recent candidates who might be looking for a project?"
Finished 15 questions...
No results for question - "I HAVE A 13 YEAR OLD ROSEBUSH I NEED TO MOVE DUE TO LACK OF FULL SUN. CAN IT BE MOVED SAFELY NOW?"
Finished 20 questions...
Finished 25 questions...
Finished 30 questions...
No results for question - "What is a good soil mix for blueberries?"
No results for question - "How do you tell when pears are ready to pick?"
Finished 35 questions...
Finished querying all (37) questions for scoring


In [20]:
no_results = 0
for i, res in enumerate(results):
    if len(res) == 0:
        print(f'------------------------------------------------------------------')
        print(f'Question number     : {i+1}')
        print(f'Question            : {questions[i]}')
        print(f'Number of results   : {len(res)}')
        no_results += 1

print(f'Out of {len(questions)} NA questions {no_results} have correctly returned 0 results')
print(f'Recall: {no_results/len(questions) * 100:.2f}%')

------------------------------------------------------------------
Question number     : 13
Question            : I'm a 4-H alumni. What is the nearest club near me? If there isn't one close, can I start my own club?
Number of results   : 0
------------------------------------------------------------------
Question number     : 15
Question            : I'm interested in hiring a master gardener from the master gardener program. Do you have recent candidates who might be looking for a project?
Number of results   : 0
------------------------------------------------------------------
Question number     : 19
Question            : I HAVE A 13 YEAR OLD ROSEBUSH I NEED TO MOVE DUE TO LACK OF FULL SUN. CAN IT BE MOVED SAFELY NOW?
Number of results   : 0
------------------------------------------------------------------
Question number     : 31
Question            : What is a good soil mix for blueberries?
Number of results   : 0
---------------------------------------------------------------

### NA questions errors

In [15]:
for i, res in enumerate(results):
    if len(res) > 0:
        print(f'------------------------------------------------------------------')
        print(f'Question number     : {i+1}')
        print(f'Question            : {questions[i]}')
        print(f'Number of results   : {len(res)}')
        for i1, res in enumerate(results[i]):
            print(f'Results ranked {i1+1:<2}   : {res[0][:30] + "...":<33} {"(URL: " + res[1]:>10s})')

------------------------------------------------------------------
Question number     : 1
Question            : my Night Queen plants have developed brown spots on the leaves and around the edges on some of the leaves. I have followed the advice on watering, light, soil etc. Also tried putting cinnamon on the affected leaves but to no avail. Should I be cutting off the affected leaves or just leave them alone? The plant is very healthy apart from the little brown spots.
Number of results   : 10
Results ranked 1    : Potassium deficiency...           (URL: http://ipm.ucanr.edu/PMG/GARDEN/PLANTS/DISORDERS/potassiumdeficiency.html?src=exchbt)
Results ranked 2    : Powdery mildew on peas and bea... (URL: http://ipm.ucanr.edu/PMG/GARDEN/VEGES/DISEASES/peapowdermildew.html?src=exchbt)
Results ranked 3    : Squash bug...                     (URL: http://ipm.ucanr.edu/PMG/GARDEN/VEGES/PESTS/squashbug.html?src=exchbt)
Results ranked 4    : Scab...                           (URL: http://ipm.uca

In [16]:
error_results = {
    'question_number'   : [],
    'question_text'     : [],
    'result_urls'       : []
}

for i in range(10):
    error_results['result_urls'].append([])


for i, res in enumerate(results):
    if len(res) > 0:
        error_results['question_number' ].append(i+1            )
        error_results['question_text'   ].append(questions[i]   )
        
        r_temp = ['No answer'] * 10
        for i1, res in enumerate(results[i]):
            r_temp[i1] = f'{res[0][:30] + "...":<33} {"(URL: " + res[1]:>10s})'
        for i1, res in enumerate(r_temp):
            error_results['result_urls'][i1].append(res)


In [17]:
data = {
    'QuestionNumber': error_results['question_number'   ],
    'QuestionText'  : error_results['question_text'     ],
    'ResultRank1'   : error_results['result_urls'][0],
    'ResultRank2'   : error_results['result_urls'][1],
    'ResultRank3'   : error_results['result_urls'][2],
    'ResultRank4'   : error_results['result_urls'][3],
    'ResultRank5'   : error_results['result_urls'][4],
    'ResultRank6'   : error_results['result_urls'][5],
    'ResultRank7'   : error_results['result_urls'][6],
    'ResultRank8'   : error_results['result_urls'][7],
    'ResultRank9'   : error_results['result_urls'][8],
    'ResultRank10'  : error_results['result_urls'][9]
}

df_errors = pd.DataFrame(data = data).set_index('QuestionNumber')
print(f'Shape of error answers dataframe: {df_errors.shape}')
df_errors.sample(10)

Shape of error answers dataframe: (32, 11)


,QuestionText,ResultRank1,ResultRank2,ResultRank3,ResultRank4,ResultRank5,ResultRank6,ResultRank7,ResultRank8,ResultRank9,ResultRank10
QuestionNumber,,,,,,,,,,,
20,Do you know an affordable source to get evergr...,Chinese pistache... (URL: http:/...,Laurel... (URL: http:/...,"Maidenhair tree, Ginkgo... (URL: http:/...",Escallonia... (URL: http:/...,Cryptomeria... (URL: http:/...,Maple... (URL: http:/...,Rose... (URL: http:/...,Australian willow... (URL: http:/...,Tree Borers... (URL: http:/...,Oak... (URL: http:/...
28,The trees on my property have long branches th...,Flannel bush... (URL: http:/...,"Maidenhair tree, Ginkgo... (URL: http:/...",Katsura tree... (URL: http:/...,First-year pruning for fruit a... (URL: http:/...,Australian willow... (URL: http:/...,Pepper tree (California pepper... (URL: http:/...,Crape myrtle... (URL: http:/...,"Araucaria, Bunya-Bunya tree, M... (URL: http:/...",Climbing cutworms... (URL: http:/...,Pruning walnut trees... (URL: http:/...
17,"I bought a farm this year, and I'm learning ab...",Pruning your potatoes... (URL: http:/...,Field Bindweed... (URL: http:/...,Powdery scab... (URL: http:/...,Pumpkins... (URL: http:/...,Fertilizing your berries... (URL: http:/...,Corn... (URL: http:/...,Cantaloupe... (URL: http:/...,Russian Thistle... (URL: http:/...,Planting cucurbits... (URL: http:/...,Verticillium wilt... (URL: http:/...
16,What are the recommended sprays to use on appl...,Pruning cherry trees... (URL: http:/...,Pruning apple and pear trees... (URL: http:/...,Scab... (URL: http:/...,Pruning apricot trees... (URL: http:/...,Fire Blight... (URL: http:/...,Training apple and pear trees... (URL: http:/...,Asian Citrus Psyllid and Huang... (URL: http:/...,"Bordeaux mixture - fungicide, ... (URL: http:/...","Cedar, cypress, and juniper ru... (URL: http:/...",Pruning peach and nectarine tr... (URL: http:/...
12,I'm looking for a planting guide on current we...,Planting... (URL: http:/...,Planting... (URL: http:/...,Planting... (URL: http:/...,Planting... (URL: http:/...,Planting... (URL: http:/...,Planting... (URL: http:/...,Planting... (URL: http:/...,Planting... (URL: http:/...,Plant properly... (URL: http:/...,Common environmental disorders... (URL: http:/...
18,What cultivar would you recommend for a good t...,Raspberry horntail... (URL: http:/...,Raspberry sawfly... (URL: http:/...,Red sorrel... (URL: http:/...,Strawberries... (URL: http:/...,Mulberry... (URL: http:/...,Elderberry... (URL: http:/...,Cherry crinkle... (URL: http:/...,Blackberries and Raspberries... (URL: http:/...,Raspberry crown borer:... (URL: http:/...,Pruning strawberries... (URL: http:/...
22,I have a few apple trees. The one has a bent o...,Pepper tree (California pepper... (URL: http:/...,"Maidenhair tree, Ginkgo... (URL: http:/...",Pruning apple and pear trees... (URL: http:/...,First-year pruning for fruit a... (URL: http:/...,Fir... (URL: http:/...,"Cedar, cypress, and juniper ru... (URL: http:/...",Katsura tree... (URL: http:/...,Bermudagrass... (URL: http:/...,"Araucaria, Bunya-Bunya tree, M... (URL: http:/...",Erythrina Stem Borer... (URL: https:...
14,We would like to prepare the soil for a vegeta...,Eggplant... (URL: http:/...,Squash... (URL: http:/...,Cabbage... (URL: http:/...,Lettuce... (URL: http:/...,Chard... (URL: http:/...,Tomato... (URL: http:/...,Spinach... (URL: http:/...,Soil preparation... (URL: http:/...,Carrot... (URL: http:/...,Pumpkins... (URL: http:/...
24,There are holes approx. 6 inches wide and 6-8 ...,Pocket Gophers... (URL: http:/...,Gophers... (URL: http:/...,Weevils... (URL: http:/...,Voles (Meadow Mice)... (URL: http:/...,Raccoons... (URL: http:/...,Skunks... (URL: http:/...,Fertilizing your strawberries... (URL: http:/...,Whiteflies... (URL: http:/...,Earwigs... (URL: http:/...,Worming... (URL: http:/...


In [18]:
ERRORS_NA_DATA = './data/errors/errors_na.csv'
df_errors.to_csv(ERRORS_NA_DATA)

In [19]:
pd.read_csv(ERRORS_NA_DATA).iloc[9]

QuestionNumber                                                   10
QuestionText      I'm interested in finding a local source for v...
ResultRank1       Invasive Plants...                (URL: http:/...
ResultRank2       Linden...                         (URL: http:/...
ResultRank3       Gardenia (Cape jasmine)...        (URL: http:/...
ResultRank4                                               No answer
ResultRank5                                               No answer
ResultRank6                                               No answer
ResultRank7                                               No answer
ResultRank8                                               No answer
ResultRank9                                               No answer
ResultRank10                                              No answer
Name: 9, dtype: object